In [1]:
import os
import json
import pickle
from bisect import bisect_left, bisect_right
from datetime import datetime, timedelta
from collections import defaultdict, Counter

import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
DATADIR = "hackathon_data" # "./data"
transactions_path = f"{DATADIR}/avk_hackathon_data_transactions.csv"
pd.read_csv(f"{DATADIR}/avk_hackathon_data_transactions.csv", nrows=5)

,party_rk,account_rk,financial_account_type_cd,transaction_dttm,transaction_type_desc,transaction_amt_rur,merchant_rk,merchant_type,merchant_group_rk,category
0,20337,19666,1,2019-01-01,Покупка,84.00,88676.0,348.0,NaN,Сувениры
1,63404,72991,1,2019-01-01,Покупка,410.00,887248.0,330.0,725.0,Фаст Фуд
2,24789,23517,2,2019-01-01,Покупка,701.44,830014.0,291.0,NaN,Супермаркеты
3,57970,64838,2,2019-01-01,Покупка,6203.70,363834.0,278.0,454.0,Дом/Ремонт
4,12232,11591,2,2019-01-01,Покупка,734.53,85919.0,286.0,878.0,Супермаркеты


In [195]:
trans = pd.read_csv(f"{DATADIR}/avk_hackathon_data_transactions.csv")

In [148]:
data_trans_train = trans[pd.to_datetime(trans.transaction_dttm) < pd.to_datetime('2019-11-01')]

In [153]:
month = []
for x in range(1,10):
    month.append(pd.to_datetime(f'2019-0{x}-01'))
for x in range(10,13):
    month.append(pd.to_datetime(f'2019-{x}-01'))

In [154]:
party_tr = data_trans_train.party_rk.unique()

In [199]:
trans.category = trans.category.fillna('<UNK>').astype(str)

In [200]:
t = trans.category

In [201]:
t.values[10:20]

array(['<UNK>', 'Супермаркеты', 'Разные товары', 'Транспорт', '<UNK>',
       '<UNK>', 'Медицинские услуги', 'Супермаркеты', 'Топливо',
       'Одежда/Обувь'], dtype=object)

In [207]:
cats = trans.category.unique()
len_cats = len(cats)
dict_cats = dict(zip(cats, range(len_cats)))

In [209]:
trans.category = [dict_cats[elem] for elem in trans.category]

In [212]:
dict_cats.values()

dict_values([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36])

In [ ]:
#count statistic for prediction

In [249]:
data_users = []
for user in party_tr:
    data_ = trans[trans.party_rk == user]
    sum_month = []
    number_cat = np.zeros(len(dict_cats.values()))
    num_month = len(month)-1
    purchase_top = []
    for i in range(num_month):
        cond1 = pd.to_datetime(data_.transaction_dttm) >= month[i]
        cond2 = pd.to_datetime(data_.transaction_dttm) < month[i+1]
        data_1 = data_[(cond1 & cond2)]
        sum_month.append(data_1.transaction_amt_rur.sum())
        cat = data_1.category.value_counts()
        for j in dict_cats.values():
            if j in cat.index:
                number_cat[j] += 1
                #print(i)
        #print(sorted(merch, key=merch.count, reverse = True))
    #print(np.array(number_cat).argsort()[-5:])
    top5 = np.array(number_cat).argsort()[-5:]
    us_top5 = []
    for i in range(num_month):
        top5_sum = []
        cond1 = pd.to_datetime(data_.transaction_dttm) >= month[i]
        cond2 = pd.to_datetime(data_.transaction_dttm) < month[i+1]
        data_1 = data_[(cond1 & cond2)]
        sum_month_1 = data_1.transaction_amt_rur.sum()
        for j in top5:
            fromtop = data_1[data_1.category == j].transaction_amt_rur.sum()
            top5_sum.append(fromtop)
            
        #print(top5_sum)
        us_top5.append(top5_sum)
    
    data_users.append([np.array(number_cat).argsort()[-5:], np.mean(np.array(us_top5), axis=0), np.std(np.array(us_top5), axis=0), sum_month ])

KeyboardInterrupt: 

In [256]:
d_len = int(len(data_users))

In [262]:
data_users = np.array(data_users)

In [273]:
def give_top_and_sum(us, top_n=5):
    cat, sum_mean, sum_std, sum_month = data_users[party_tr[:d_len] == us][0]
    cat, sum_mean, sum_std = cat[-top_n:], sum_mean[-top_n:], sum_std[-top_n:]
    limits = sum_mean + 3 * sum_std 
    other_lim = np.mean(sum_month) - sum(sum_mean)
    if other_lim < 0:
        other_lim = sum(sum_std)
    return limits, other_lim, cat

In [286]:
#check our model
i=0
users = []
#choose active users
for user in party_tr[:d_len]:
    user_lim = 1000
    cat, sum_mean, sum_std, sum_month = data_users[party_tr[:d_len] == user][0]
    if sum(sum_month) > 10000:
        users.append(user)
        i += 1
    if i == user_lim:
        break
for user in users:
    data_ = trans[trans.party_rk == user]
    limits, other_lim, cat = give_top_and_sum(user, 5)
    cond1 = pd.to_datetime(data_.transaction_dttm) >= month[-1]
    data_1 = data_[cond1]
    err = []
    abs_err = []
    all_real = []
    user_bad = 0
    for i, cat_ in enumerate(cat):
        real = data_1[data_1.category == cat_].transaction_amt_rur.sum()
        if(real == 0):
            user_bad = 1
        all_real.append(real)
        err_lim = real - limits[i]
        abs_e = abs(err_lim)
        if(err_lim < 0):
            err_lim = 0
        err.append(err_lim)
        abs_err.append(abs_e)
    other = data_1.transaction_amt_rur.sum() - sum(all_real)
    err_oth = other - other_lim
    err.append(err_oth)
    if(user_bad == 0 and sum(abs_err)<20000):
        print("User ", user, sum(abs_err), abs_err, err, cat, limits, other_lim, all_real, other)

User  11619 16304.304885052969 [9005.22356274439, 686.7836403635438, 2622.2938666975624, 1692.729218836561, 2297.2745964109126] [0, 0, 2622.2938666975624, 0, 0, 2640.22818181818] [ 7 13  2  6  3] [ 9138.22356274  1536.78364036  7828.2061333  54548.87921884
 14507.79459641] 2134.4118181818194 [133.0, 850.0, 10450.5, 52856.149999999994, 12210.52] 4774.639999999999
User  43121 16792.595284982406 [4042.111382452239, 954.8068748323361, 2823.3629845030982, 7227.856651478167, 1744.4573917165653] [4042.111382452239, 0, 2823.3629845030982, 7227.856651478167, 0, -33.99363636363205] [12  1  6 15  2] [ 753.88861755 1009.80687483 3869.6370155  7744.14334852 4298.16739172] 208.9936363636357 [4796.0, 55.0, 6693.0, 14972.0, 2553.71] 175.00000000000364
User  4412 17497.750743042227 [2161.3320926990737, 7562.088368904391, 2675.893486303106, 4397.7314122339685, 700.7053829016877] [0, 0, 0, 0, 700.7053829016877, -219.18181818181802] [ 7 16  1  2  6] [ 2308.8320927  10136.0883689   3162.8934863   7907.1314

As an example: 

user 47416 have predicted limits in 5 categories:  1713.73087026  5435.07515136  6295.51170484  2470.26636864
 68751.56530765
real spending: 513.0, 1625.4, 2472.3500000000004, 512.0, 73617.0
for other spending predicted: 7711.567272727272, real: 28588.98999999999
What we have: good limits for top categories, prediction for other spending could be better

In [268]:
data_users[party_tr[:d_len] == 20337][0][0]

array([ 1,  2, 15,  6, 22])

In [261]:
party_tr[:d_len] == 20337

array([ True, False, False, ..., False, False, False])

In [ ]:
#categorize the users

In [126]:
soc = pd.read_csv(f"{DATADIR}/avk_hackathon_data_party_x_socdem.csv")
soc

,party_rk,gender_cd,age,marital_status_desc,children_cnt,region_flg
0,61243,F,70.0,NaN,0,0
1,66535,F,25.0,NaN,0,0
2,83721,M,55.0,Женат/замужем,0,0
3,88238,F,35.0,NaN,0,0
4,57179,F,30.0,Женат/замужем,0,0
...,...,...,...,...,...,...
49995,54994,M,40.0,Женат/замужем,0,0
49996,63391,M,35.0,Холост/не замужем,0,0
49997,5418,F,55.0,Холост/не замужем,0,0
49998,50273,F,35.0,Женат/замужем,0,0


In [127]:
cond1 = soc.gender_cd != 'F'
cond2 = soc.gender_cd != 'M'
soc[cond1 & cond2]

,party_rk,gender_cd,age,marital_status_desc,children_cnt,region_flg
1285,52096,NaN,30.0,NaN,0,0
1616,62173,NaN,25.0,Холост/не замужем,0,0
5534,78758,NaN,25.0,Гражданский брак,0,0
10379,86443,NaN,45.0,Разведен (а),0,0
15382,94159,NaN,40.0,Женат/замужем,2,0
20075,49175,NaN,25.0,NaN,0,0
24240,63068,NaN,55.0,NaN,0,0
25927,55736,NaN,45.0,Женат/замужем,0,0
41774,66523,NaN,35.0,NaN,0,0
47842,53585,NaN,70.0,NaN,4,1


In [128]:
soc.marital_status_desc = soc.marital_status_desc.fillna("<UNK>").astype(str)

In [129]:
def mapping(x, d):
    for key, val in zip(d.keys(), d.values()):
        if x <= val:
            return key
        
def mapping1(x, d):
    for key, val in zip(d.keys(), d.values()):
        if x == key:
            return val


In [130]:
d_age = {1: 20, 2: 30, 3: 45, 4: 65, 5: 100}
d_chi = {1: 0, 2: 3}
d_gen = {"F": 0, "M": 1}
soc['age_map'] = [mapping(i, d_age) for i in soc.age]
soc['chi_map'] = [mapping(i, d_chi) for i in soc.children_cnt]
soc['gen_map'] = [mapping1(i, d_gen) for i in soc.gender_cd]

In [131]:
d_mar = {"Женат/замужем": 1, "Гражданский брак": 1, "Разведен (а)": 2, "Холост/не замужем": 3, "Вдовец, вдова" :2, "Не проживает с супругом (ой)":2, "<UNK>": 4}
soc['mar_map'] = [mapping1(i, d_mar) for i in soc.marital_status_desc]

In [132]:
m = soc.groupby(['chi_map', 'region_flg', 'gen_map', 'age_map', 'mar_map'])['party_rk'].mean()

In [133]:
m = m.reset_index()
m['class'] = range(1,132)
m = m.drop(columns=['party_rk'])

In [134]:
soc = soc.merge(m, how='left', on=['chi_map', 'region_flg', 'gen_map', 'age_map', 'mar_map'])

In [142]:
soc[['party_rk','class']].to_csv("classes.csv")

In [22]:
d_age = {1: 20, 2: 30, 3: 45, 4: 65, 5: 100}
d_age.values()

dict_values([20, 30, 45, 65, 100])

In [25]:
for key, val in zip(d_age.keys(), d_age.values()):
    print(key, val)

1 20
2 30
3 45
4 65
5 100
